In [5]:
import numpy as np
# import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt 
import pickle 
import json

Loading the dataset

In [10]:
# ruta = '../datasets/'
# df_0 = pd.read_csv(ruta+'simpsons/simpsons_dataset.csv').dropna()
# target_simpsons = [
#     'Lisa Simpson',
#     'Bart Simpson',
#     'Homer Simpson',
#     'Marge Simpson',
#     'Maggie Simpson',
#     'Grampa Simpson'
# ]
# data = df_0.loc[df_0['raw_character_text'].isin(target_simpsons)]
# data.head(5)

In [6]:
english_path = "../../documents/en.jsonl"

file = open(english_path, "r")
data = []
for index, line in enumerate(file):
    data.append(json.loads(line))

text_raw_df = pd.json_normalize(data)

In [12]:
text_raw_df

,index,author_id,text,lang,context_annotations,id,created_at,date_str,hashtag,tag
0,1169503,117418104,#Bucks: Milwaukee Bucks and Milwaukee Health D...,en,"[{'domain': {'id': '3', 'name': 'TV Shows', 'd...",1387163086981898240,2021-04-27T21:53:46.000Z,2021-04-27,[Bucks],vaccination
1,1088588,726456608,Youth Ages 12-15 Now Eligible for Pfizer COVID...,en,"[{'domain': {'id': '65', 'name': 'Interests an...",1392964893469511681,2021-05-13T22:08:05.000Z,2021-05-13,[],vaccination
2,1229144,191092262,Fighting Stigma: Young people should be free t...,en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1387110938088677376,2021-04-27T18:26:33.000Z,2021-04-27,[],vaccination
3,1041354,2985792143,Second jab done! Although by the state of Tesc...,en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1391014287565545477,2021-05-08T12:57:04.000Z,2021-05-08,[],school-reopening
4,1190556,19809471,4/27 - Vaccination and Testing Update - 2020-2...,en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1387142140606648320,2021-04-27T20:30:32.000Z,2021-04-27,[],vaccines
...,...,...,...,...,...,...,...,...,...,...
5864,796383,1371495185767526413,"families are burdened, students are burned out...",en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1379007009454911492,2021-04-05T09:44:26.000Z,2021-04-05,[],school-reopening
5865,779950,203824292,What happens when everyone who is now vaccinat...,en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1379030125191254018,2021-04-05T11:16:17.000Z,2021-04-05,"[COVIDー19, Coronavirus, VaccinePassports]",vaccination
5866,680034,429541970,duh. some countries already do...ha!\nDufus Dr...,en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1379120339943268354,2021-04-05T17:14:46.000Z,2021-04-05,"[vaccine, vaccination]",NONE
5867,382165,1154783639378583553,"A year into pandemic, professor mental health ...",en,"[{'domain': {'id': '65', 'name': 'Interests an...",1379163452166578187,2021-04-05T20:06:05.000Z,2021-04-05,[mentalhealth],mental-health


In [13]:
sentences = text_raw_df['text'].values
y = text_raw_df['tag'].values

Splitting the dataset

In [14]:
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.5, random_state=1000)

Vectorize the dataset

In [15]:
top_words = 5000
vectorize_layer = TextVectorization(
    ngrams=None, max_tokens=top_words, vocabulary=None,
    output_mode='int', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer.adapt(sentences_train)
print(len(vectorize_layer.get_vocabulary()))

5000


In [16]:
x_train = vectorize_layer(np.array([[s] for s in sentences_train])).numpy()
x_test = vectorize_layer(np.array([[s] for s in sentences_test])).numpy()

y_train = np.array(y_train)
y_test = np.array(y_test)

In [17]:
# (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=top_words)
# Parece que te tira el dataset como 25 mil train, 25 mil test, con cada entrada el embedding 'int' del doc.
# Esto nos tocaría usar un vectorizze layer sobre nuestros docs con output mode 'int'


In [18]:
# len(X_train[3])
print(type(x_train))
len(x_test)
print(len(y_test))

<class 'numpy.ndarray'>
2935


Pad or truncate all sequences to half of max lenght

In [19]:
# Podriamos usar esto o simplemente definir un número 'bueno' y truncar o paddear todo a ese número
# Con twitter depronto sí sirve un número de 25 por ejemplo.
max_doc_length = np.max([len(doc) for doc in x_train])

print('Máximo tamaño de documento en train:', max_doc_length)
x_train_padded = sequence.pad_sequences(x_train, maxlen=(max_doc_length))
x_test_padded = sequence.pad_sequences(x_test, maxlen=(max_doc_length))

Máximo tamaño de documento en train: 94


Enconding classification classes

In [20]:
encoder = LabelBinarizer()

encoder.fit(y_train)
y_train_bin = encoder.transform(y_train)
y_test_bin = encoder.transform(y_test)

In [21]:
num_classes = len(encoder.classes_)
print('Número de clases:', num_classes)

Número de clases: 6


## Model definition

In [22]:
# Acá hay que tener cuidado porque nuestro problema de clasificación No es binario
# Con los datos reales el Compile debe ser con 'categorical_crossentropy'
# Además el tamaño de la última softmax debe ser el mismo que el número de categorias
# Se tiene también que pasar los datos en Y a oneHotvectors, ese código lo podemos sacar del notebool model_V1

# create the model
embedding_vecor_length = 256
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_doc_length))
model.add(LSTM(100))
model.add(Dense(num_classes, activation='softmax'))

Compilation

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 94, 256)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               142800    
_________________________________________________________________
dense (Dense)                (None, 6)                 606       
Total params: 1,423,406
Trainable params: 1,423,406
Non-trainable params: 0
_________________________________________________________________
None


Training

In [24]:
model.fit(x_train_padded, y_train_bin, validation_data=(x_test_padded, y_test_bin), epochs=5, batch_size=64)

Epoch 1/5
46/46 [==============================] - 5s 85ms/step - loss: 1.5466 - accuracy: 0.4268 - val_loss: 1.4131 - val_accuracy: 0.4388
Epoch 2/5
46/46 [==============================] - 3s 73ms/step - loss: 1.4209 - accuracy: 0.4501 - val_loss: 1.4043 - val_accuracy: 0.4388
Epoch 3/5
46/46 [==============================] - 3s 72ms/step - loss: 1.4136 - accuracy: 0.4488 - val_loss: 1.4103 - val_accuracy: 0.4388
Epoch 4/5
46/46 [==============================] - 3s 73ms/step - loss: 1.4210 - accuracy: 0.4418 - val_loss: 1.4075 - val_accuracy: 0.4388
Epoch 5/5
46/46 [==============================] - 3s 73ms/step - loss: 1.4127 - accuracy: 0.4426 - val_loss: 1.4065 - val_accuracy: 0.4388


In [ ]:
from sklearn.metrics import classification_report
#from sklearn.metrics import confusion_matrix


def get_metrics_by_class(model, x, y):
  y_pred = model.predict(x, batch_size=64, verbose=1)
  y_pred_bool = np.argmax(y_pred, axis=1)
  y_label = np.argmax(y, axis=1)
  #print(confusion_matrix(y_pred_bool, y_label))
  print(classification_report(y_label, y_pred_bool))